<img src="../PythonBootcamp/support_files/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Introduction to the Allen Mouse Brain Connectivity Atlas</h1> 
<h3 align="center">August 23, 2016</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>This notebook demonstrates how to access and manipulate data in the Allen Mouse Brain Connectivity Atlas.
</div>

In [ ]:
# please make sure your drive_path is set, so that the notebook can find the data files on the hard drive

# OS X
drive_path = '/Volumes/Brain2016'

# Windows (a good guess)
# drive_path = 'e:/'

# Linux (will vary; the following is possibly what Ubuntu will do)
# drive_path = '/media/Brain2016/'

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>The `MouseConnectivityCache` AllenSDK class provides methods for downloading metadata about experiments, including their viral injection site and the mouse's transgenic line. You can request information either as a Pandas DataFrame or a simple list of dictionaries.

<p>An important feature of the `MouseConnectivityCache` is how it stores and retrieves data for you. By default, it will create (or read) a manifest file that keeps track of where various connectivity atlas data are stored. If you request something that has not already been downloaded, it will download it and store it in a well known location.
</div>

In [ ]:
import os
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [ ]:
# When downloading 3D connectivity data volumes, what resolution do you want (in microns)?  
# Options are: 10, 25, 50, 100
resolution_um=25

# The manifest file is a simple JSON file that keeps track of all of
# the data that has already been downloaded onto the hard drives.
# If you supply a relative path, it is assumed to be relative to your
# current working directory.
manifest_file = os.path.join(drive_path, "MouseConnectivity","manifest.json")

mcc = MouseConnectivityCache(manifest_file=manifest_file, resolution=resolution_um)

# open up a pandas dataframe of all of the experiments
all_experiments = mcc.get_experiments(dataframe=True)
print "%d total experiments" % len(all_experiments)

# take a look at what we know about an experiment with a primary motor injection
all_experiments.loc[122642490]

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 1:**
<ol>
<li>Go to the mouse connectivity atlas (http://connectivity.brain-map.org)
<li>Find an experiment ID for an injection in your favorite structure.  
<li>Note this experiment's transgenic line.
<li>Verify that the contents of `all_experiments` for that experiment are consistent with what's on the web application.
</ol>   
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>`MouseConnectivityCache` has a method for retrieving the adult mouse structure ontology as an `Ontology` class instance. This is a light wrapper around a Pandas DataFrame of structures that provides some helper methods for figuring out which structures are parents/children of each other. It's also handy for finding the ID of a structure by its acronym.
</div>

In [ ]:
# grab the Ontology instance
ontology = mcc.get_ontology()

# get some info on the isocortex
isocortex = ontology['Isocortex']
isocortex

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.1:**  Take a look at what functions are available in the Ontology class using 'help' or 'dir'
 </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.2:**  Print the name of the parent structure of isocortex.  
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
**Exercise 2.3:**  Figure out the acronym for secondary motor area.  Hint: http://connectivity.brain-map.org, http://atlas.brain-map.org
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.4:** Print the acronyms for all of the visual areas in the ontology.
 </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.5:** Print a list of acronyms for descendant structures of the cerebellum.
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>On the connectivity atlas web site, you'll see that we show most of our data at a fairly coarse structure level. We did this by creating a "structure set" of ~300 structures. If you want to filter your ontology structures down to that list, you can do this:
</div>

In [ ]:
from allensdk.api.queries.ontologies_api import OntologiesApi

# get the adult mouse structures summary structures
oa = OntologiesApi()
set_name = [oa.quote_string('Mouse Connectivity - Summary')]
summary_structures = oa.get_structures(structure_set_names=set_name)

summary_structure_ids = [ s['id'] for s in summary_structures ]

# let's just look at the first five
ontology[summary_structure_ids[:5]]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>This is how you can filter experiments by transgenic line:
</div>

In [ ]:
# fetch the experiments that have injections in the isocortex of cre-positive mice
cre_cortical_experiments = mcc.get_experiments(cre=True, 
                                                injection_structure_ids=isocortex['id'])

print("%d Cre cortical experiments" % len(cre_cortical_experiments))

# same as before, but restrict the cre line
rbp4_cortical_experiments = mcc.get_experiments(cre=[ 'Rbp4-Cre_KL100' ], 
                                                injection_structure_ids=isocortex['id'])


print("%d Rbp4 cortical experiments" % len(rbp4_cortical_experiments))

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 3.1:**  Make a unique list of all Cre lines associated with any experiment.
  
   </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 3.2:**  Get all the wild-type experiments delivered to VISp. How many are there? How are they the same and different? 
   </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 3.3:**  Pick your favorite Cre line (or use the one above). Count the injections with this line targeted to VISp. Repeat the analysis above.
   </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 3.4:**  Figure out how many experiments in each Cre line are available in VISp.
   </div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Structure Signal Unionization</h2>

<p>The ProjectionStructureUnionizes API data tells you how much signal there was in a given structure and experiment. It contains the density of projecting signal, volume of projecting signal, and other information. `MouseConnectivityCache` provides methods for querying and storing this data.

<p>**Important structure unionize columns**

<ul>
<li> projection_density: sum of detected pixels / volume of structure   
<li> projection_energy: sum of detected pixel intensity / volume of structure
<li> projection_volume: volume of projecting signal in mm3
<li> normalized_projection_volume: projection_volume / volume of injection site
<li> is_injection: 
    <ul>
   <li> True => record contains signal only inside injection site
   <li> False => record contains signal only outside injection site
   </ul>
<li> hemisphere_id:
    <ul>
  <li> 1 => left hemisphere only
  <li> 2 => right hemisphere only
  <li> 3 => both hemispheres
  </ul>
</ul>

<p>Please see http://help.brain-map.org/display/mouseconnectivity/API#API-ProjectionDataGridding for more information.
</div>

In [ ]:
# find wild-type injections into primary visual area
visp = ontology['VISp']
visp_experiments = mcc.get_experiments(cre=False, 
                                       injection_structure_ids=visp['id'])

print("%d VISp experiments" % len(visp_experiments))

structure_unionizes = mcc.get_structure_unionizes([ e['id'] for e in visp_experiments ], 
                                                  is_injection=False,
                                                  structure_ids=isocortex['id'])

print("%d VISp non-injection, cortical structure unionizes" % len(structure_unionizes))

In [ ]:
structure_unionizes.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>This is a rather large table, even for a relatively small number of experiments.  You can filter it down to a smaller list of structures like this.
</div>

In [ ]:
dense_unionizes = structure_unionizes[ structure_unionizes.projection_density > .5 ]
large_unionizes = dense_unionizes[ dense_unionizes.volume > .5 ]
large_structures = ontology[large_unionizes.structure_id]

print("%d large, dense, cortical, non-injection unionizes, %d structures" % ( len(large_unionizes), len(large_structures) ))

print(large_structures.name)

large_unionizes

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 4.1:**  Get the StructureUnionize records for injections into primary motor areas, restricted to target structures in the hippocampal formation.  
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 4.2:**  Restrict further to only HPF, not all descendants.
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 4.3:**  Print the top 5 records by projection signal density.
  </div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Generating a Projection Matrix</h2>
<p>The `MouseConnectivityCache` class provides a helper method for converting ProjectionStructureUnionize records for a set of experiments and structures into a matrix.  This code snippet demonstrates how to make a matrix of projection density values in auditory sub-structures for cre-negative VISp experiments. 
</div>

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


visp_experiment_ids = [ e['id'] for e in visp_experiments ]
ctx_children = ontology.get_child_ids( ontology['Isocortex'].id )

pm = mcc.get_projection_matrix(experiment_ids = visp_experiment_ids, 
                               projection_structure_ids = ctx_children,
                               hemisphere_ids= [2], # right hemisphere, ipsilateral
                               parameter = 'projection_density')

row_labels = pm['rows'] # these are just experiment ids
column_labels = [ c['label'] for c in pm['columns'] ] 
matrix = pm['matrix']

fig, ax = plt.subplots(figsize=(15,15))
heatmap = ax.pcolor(matrix, cmap=plt.cm.afmhot)

# put the major ticks at the middle of each cell
ax.set_xticks(np.arange(matrix.shape[1])+0.5, minor=False)
ax.set_yticks(np.arange(matrix.shape[0])+0.5, minor=False)

ax.set_xlim([0, matrix.shape[1]])
ax.set_ylim([0, matrix.shape[0]])          

# want a more natural, table-like display
ax.invert_yaxis()
ax.xaxis.tick_top()

ax.set_xticklabels(column_labels, minor=False)
ax.set_yticklabels(row_labels, minor=False)
plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 5.1:**  Make a projection matrix using the same experiments from exercise 4, but for left-hemisphere data in children of isocortex.
       </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 5.2:**  Plot the projection matrix as a heat map.
       </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 5.3:**  Advanced: try to build the projection matrix without using get_projection_matrix.
<ol>
<li>retrieve the structure unionize records described in 5.1
<li>pivot the records into a projection density matrix
       </ol>
       </div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Manipulating Grid Data</h2>

<p>The `MouseConnectivityCache` class also helps you download and open every experiment's projection grid data volume. By default it will download 25um volumes, but uou could also download data at other resolutions if you prefer (10um, 50um, 100um).

<p>This demonstrates how you can load the projection density for a particular experiment. It also shows how to download the template volume to which all grid data is registered. Voxels in that template have been structurally annotated by neuroanatomists and stored in a separate annotation volume image.
</div>

In [ ]:
experiment_id = 181599674

# projection density: number of projecting pixels / voxel volume
pd, pd_info = mcc.get_projection_density(experiment_id)

# injection density: number of projecting pixels in injection site / voxel volume
ind, ind_info = mcc.get_injection_density(experiment_id)

# injection fraction: number of pixels in injection site / voxel volume
inf, inf_info = mcc.get_injection_fraction(experiment_id)

# data mask:
# binary mask indicating which voxels contain valid data
dm, dm_info = mcc.get_data_mask(experiment_id)

template, template_info = mcc.get_template_volume()
annot, annot_info = mcc.get_annotation_volume()

print(pd_info)
print(pd.shape, template.shape, annot.shape)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>Once you have these loaded, you can use matplotlib see what they look like.
</div>

In [ ]:
# compute the maximum intensity projection of the projection density
pd_mip = pd.max(axis=0)
ind_mip = ind.max(axis=0)
inf_mip = inf.max(axis=0)
dm_mip = dm.min(axis=0)

# show that slice of all volumes side-by-side
f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(15, 6))

ax1.imshow(pd_mip, cmap='hot', aspect='equal')
ax1.set_title("projection density MaxIP")

ax2.imshow(ind_mip, cmap='hot', aspect='equal')
ax2.set_title("injection density MaxIP")

ax3.imshow(inf_mip, cmap='hot', aspect='equal')
ax3.set_title("injection fraction MaxIP")

# pick a slice to show
slice_idx = 26
ax4.imshow(template[slice_idx,:,:], cmap='gray', aspect='equal', vmin=template.min(), vmax=template.max())
ax4.set_title("registration template")

ax5.imshow(annot[slice_idx,:,:], cmap='gray', aspect='equal', vmin=0, vmax=2000)
ax5.set_title("annotation volume")

ax6.imshow(dm_mip, cmap='hot', aspect='equal', vmin=0, vmax=1)
ax6.set_title('data mask MinIP')

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>The `MouseConnectivityCache` also provides you with a way to compute and store structure masks. A structure mask is a numpy array that tells you if a voxel belongs to a particular structure.

<p>How is this different from the annotation volume? The annotation volume assigns each voxel a single label, usually one that is at the bottom of the structure ontology. You won't find a single voxel labeled 'Isocortex' in the annotation volume, but there are many voxels that are labeled as descendants of the Isocortex. `MouseConnectivityCache` can compute for you a mask of voxels that are labeled as any of the descendants of a structure.

<p>Careful.  Computing masks is a time-consuming task!
</div>

In [ ]:
isocortex_mask, _ = mcc.get_structure_mask(isocortex['id'])

# pull out the values of all voxels in the isocortex mask
isocortex_pd = pd[np.where(isocortex_mask)]

# print out the average projection density of voxels in the isocortex
print(isocortex_pd.mean())

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 6.1:**  Get the mask for the whole brain ('grey').
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 6.2:**  Open the projection density volumes for two experiments.
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 6.3:**  Extract the values within the brain from both experiments.
  </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise Set 6.4:**  Compute the correlation coefficient between the two.
  </div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Homework</h2>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Homework 1:**  Explain what StructureUnionize means in words. How are these data defined in terms of the data at the voxel level? Write an equation.  Start here: <link to whitepaper>.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Homework 2:**  See if you can recover the structure unionize data from the grid data for an experiment.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Homework 3:**
Think about how the structure unionize data could be used to create a connectome. See Oh et al., 2014 Supplemental Material. 
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Homework 4:**  Structure-based correlation search
<ul>
<li>Pick several experiments delivered into MOp.
<li>Get the projection density values for all structures at the "Summary" level
<li>Build an experiment-experiment correlation matrix based on those values.
</ul>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Homework 5:**  Voxel-based correlation search
<ul>
<li>Pick several experiments delivered into VISp. 
<li>Grab their projection density voxel data at 100 $\mu$m resolution.
<li>Compute an experiment-experiment correlation matrix based from those volumes.
<li>Restrict your correlation to VISp only.
</ul>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Homework 6:**  Use the Oh et al., 2014 method to fit a connectome between the visual data (or the whole brain, if you're ambitious!). For regression, look at the package scikit-learn. (Advanced)
</div>